In [1]:
!pip install streamlit faiss-cpu sentence-transformers pyngrok pandas numpy


In [2]:
with open("app.py", "w") as f:
    f.write('''
import streamlit as st
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Set page layout
st.set_page_config(page_title="Hadith Chatbot", layout="centered")
st.title("📖 Hadith Chatbot")
st.write("Ask a question and get semantically similar Hadiths.")

# Load all resources directly (no caching to avoid frozen module errors)
try:
    hadith_df = pd.read_csv("cleaned_hadith_data.csv")
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    embeddings = np.load('hadith_embeddings.npy')
    faiss_index = faiss.read_index("faiss_index.faiss")
except Exception as e:
    st.error(f"❌ Failed to load files: {e}")
    st.stop()

# Function to get similar hadiths
def get_similar_hadith(query, top_k=5):
    query_embedding = model.encode([query])
    distances, indices = faiss_index.search(np.array(query_embedding), top_k)
    results = []
    for i in range(top_k):
        idx = indices[0][i]
        row = hadith_df.iloc[idx]
        results.append({
            "Hadith": row['English_Hadith'],
            "Grade": row['English_Grade'],
            "Chapter": row['Chapter_English'],
            "Score": distances[0][i]
        })
    return results

# UI Input
query = st.text_input("🔍 Type your question:")
if query:
    st.write("🔎 Searching...")
    results = get_similar_hadith(query)
    for i, r in enumerate(results, 1):
        st.markdown(f"### 📌 Hadith {i}")
        st.markdown(f"**Chapter:** {r['Chapter']}")
        st.markdown(f"**Grade:** {r['Grade']}")
        st.markdown(f"**Similarity Score:** {r['Score']:.4f}")
        st.markdown(f"**Text:** {r['Hadith']}")
        st.markdown("---")
''')


In [4]:
!ngrok config add-authtoken 2wetnkigpK6XXRJmoNRI95vPuOv_7qfdLAYvQv8Ean6kgAWjA


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
from pyngrok import ngrok
import subprocess
import time

# Kill previous servers
!pkill streamlit > /dev/null 2>&1

# Connect ngrok, explicitly specifying http
public_url = ngrok.connect(8501, proto="http") # change is on this line
print(f"✅ Streamlit is live at: {public_url}")

# Start Streamlit app
process = subprocess.Popen(["streamlit", "run", "app.py"])
time.sleep(5)

✅ Streamlit is live at: NgrokTunnel: "https://4d65-34-74-238-55.ngrok-free.app" -> "http://localhost:8501"
